### Setup

In [1]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
#%matplotlib inline 
import numpy as np
import urllib2
import scipy.stats as stats

np.set_printoptions(precision=3, suppress=True)
url = ('https://raw.githubusercontent.com/Upward-Spiral-Science'
       '/data/master/syn-density/output.csv')
data = urllib2.urlopen(url)
csv = np.genfromtxt(data, delimiter=",")[1:] # don't want first row (labels)

# chopping data based on thresholds on x and y coordinates
x_bounds = (409, 3529)
y_bounds = (1564, 3124)

def check_in_bounds(row, x_bounds, y_bounds):
    if row[0] < x_bounds[0] or row[0] > x_bounds[1]:
        return False
    if row[1] < y_bounds[0] or row[1] > y_bounds[1]:
        return False
    if row[3] == 0:
        return False
    
    return True

indices_in_bound, = np.where(np.apply_along_axis(check_in_bounds, 1, csv,
                                                 x_bounds, y_bounds))
data_thresholded = csv[indices_in_bound]
n = data_thresholded.shape[0]


def synapses_over_unmasked(row):
    s = (row[4]/row[3])*(64**3)
    return [row[0], row[1], row[2], s]

syn_unmasked = np.apply_along_axis(synapses_over_unmasked, 1, data_thresholded)
syn_normalized = syn_unmasked

### Imaging Cortical Layers

In [7]:
# Looking at images across y, and of the layers in the y-direction
#########################################################################################
from image_builder import get_image

xs = np.unique(data_thresholded[:,0])
ys = np.unique(data_thresholded[:,1])

# Layer across y
get_image((0,1),(0,len(ys)-1),xs,ys, "across_y")

In [ ]:
# Each y-layer defined by bounds of local minima in total syn density at each y
y_bounds = [(1564,1837), (1837,2071), (2071,2305), (2305,2539), (2539,3124)]

for _, bounds in enumerate(y_bounds):
	y_lower = np.where(ys==bounds[0])[0][0]
	y_upper = np.where(ys==bounds[1])[0][0]
	print y_lower,y_upper
	i = get_image((0,1),(y_lower,y_upper),xs,ys,str(bounds[0])+"_"+str(bounds[1]))

### Held at one x-value, spanning across all Y
<img src="across_y.bmp">
### Y Idcs 1564:1837
<img src="1564_1837.bmp">
### Y Idcs 1837:2071
<img src="1837_2071.bmp">
### Y Idcs 2071:2305
<img src="2071_2305.bmp">
### Y Idcs 2305:2539
<img src="2305_2539.bmp">
### Y Idcs 2539:3124
<img src="2539_3124.bmp">

### How can we differentiate synapses from other features in the images above?

Using the NeuroDataViz tool, we looked at the kasthuri11_synapse_subcell dataset to see what we expect synapses to look like in our images (http://viz.neurodata.io/project/kasthuri11_synapse_subcell/5/367/549/1100/#). This dataset allows for special highlighting of synapses. We can see what synapses look like, and hopefully be able to make some sense of our images above.

### NeuroDataViz Screenshots. 
#### Synapses highlighted.
<img src="kasthuri11_synapse_subcell_capture.PNG">
#### Synapses highlighted, ventricles colored.
<img src="kasthuri11_synapse_subcell_capture_with_ventricles.PNG">

It looks like synapses typically occur on the edges of the ventricle-filled compartments. Can we see these ventricle-filled comparments in the images of our dataset?

<img src="ourdata_capture.png">

Using what we know about synapses from the other data images with synapses highlighted, we highlighted regions where we might expect to see synapses. 

Now, looking at the image that spans across the y layer, let's see if we find anything interesting.

Synapses seem to always appear on the edges of a ventricle compartment. There are ventricle compartments in the kasthuri11_synapse_subcell dataset that don't have synapses on the edges, but a majority of the observed region do. Using this, we expect the number of compartments fileld with ventricles to be correlated to the number of synapses in that region.

I am not able to see any strong clues for the layers across the y-direction in my images. This may be because our analysis included the sum of the whole x-z plane at a given y to produce the layer-like pattern across the y. Here, I'm only looking at one x-value and not a larger range. It may be hard to notice patterns in a sample this small.

The lower density y-layers do have more "white space" than the higher layers, and maybe this has to do with the number of ventricle-filled compartments in each subsection across the y.

All of the compartments in general seem to be smaller and more tightly packed in the higher-density layers across subsections of y. More compartments in general may mean a higher likelihood of seeing a ventricle-filled compartment and thus more synapses per area.